In [363]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from scipy import stats


### Data Preparation

In [364]:
train=pd.read_csv("C:/Users/karth/Documents/Kaggle/Purchase Prediction/train.csv")
test=pd.read_csv("C:/Users/karth/Documents/Kaggle/Purchase Prediction/test_v2.csv")
print(train.shape)
print(test.shape)

frames = [train, test]
my_data = pd.concat(frames,keys=['train', 'test'])
print(my_data.shape)


my_data['location'] = my_data['location'].fillna(0)
my_data['C_previous'] = my_data['C_previous'].fillna(99)
my_data['duration_previous'] = my_data['duration_previous'].fillna(99)
my_data['car_value'] = my_data['car_value'].fillna(99)

my_data= my_data.dropna(axis='columns')
my_data.isnull().sum(axis = 0)

(665249, 25)
(198856, 25)
(864105, 25)


customer_ID          0
shopping_pt          0
record_type          0
day                  0
time                 0
state                0
location             0
group_size           0
homeowner            0
car_age              0
car_value            0
age_oldest           0
age_youngest         0
married_couple       0
C_previous           0
duration_previous    0
A                    0
B                    0
C                    0
D                    0
E                    0
F                    0
G                    0
cost                 0
dtype: int64

In [365]:
def one_hot(df, cols):
    for col in cols:
        df[col] = df[col].astype('category')
    df=pd.concat([df,pd.get_dummies(df[cols])], axis=1)
    df.drop(cols, axis=1, inplace=True)
    return df
        
def time_convert(df):
    df['time'] = df['time'].apply(lambda x: (int(x[:2]) * 60) + int(x[3:]))
    df['time_sin'] = np.sin((df['time'] * np.pi) / 1800)
    df['time_cos'] = np.cos((df['time'] * np.pi) / 1800)
    df = df.drop(['time'], axis=1)
    return df

cat_cols = ['shopping_pt','day', 'state',
       'group_size', 'homeowner', 'car_age','car_value', 'age_oldest', 'age_youngest',
       'married_couple','C_previous','duration_previous']
my_data = one_hot(my_data,cat_cols)
my_data = time_convert(my_data)

#Split data back in to train & Test

my_train=my_data.loc['train']
my_test=my_data.loc['test']
print(my_train.shape)
print(my_test.shape)

(665249, 295)
(198856, 295)


In [366]:
#Getting the purchased Data as columns in my train
purchased_data=my_train.loc[my_train.groupby('customer_ID').record_type.idxmax()][['customer_ID','A','B','C','D','E','F','G']]
purchased_data=purchased_data.rename(columns={"A": "A_purchased","B": "B_purchased","C": "C_purchased","D": "D_purchased","E": "E_purchased","F": "F_purchased","G": "G_purchased",})
# my_train=my_train.merge(purchased_data, left_on='customer_ID', right_on='customer_ID', how='left')

In [367]:
#Getting the previous quotes as columns

# For Train Data

for i in range(1,3):
    my_train['A_previous_{}'.format(i)]=my_train['A'].shift(i)
    my_train['B_previous_{}'.format(i)]=my_train['B'].shift(i)
    my_train['C_previous_{}'.format(i)]=my_train['C'].shift(i)
    my_train['D_previous_{}'.format(i)]=my_train['D'].shift(i)
    my_train['E_previous_{}'.format(i)]=my_train['E'].shift(i)
    my_train['F_previous_{}'.format(i)]=my_train['F'].shift(i)
    my_train['G_previous_{}'.format(i)]=my_train['G'].shift(i)
    
# For Test Data

for i in range(1,3):
    my_test['A_previous_{}'.format(i)]=my_test['A'].shift(i)
    my_test['B_previous_{}'.format(i)]=my_test['B'].shift(i)
    my_test['C_previous_{}'.format(i)]=my_test['C'].shift(i)
    my_test['D_previous_{}'.format(i)]=my_test['D'].shift(i)
    my_test['E_previous_{}'.format(i)]=my_test['E'].shift(i)
    my_test['F_previous_{}'.format(i)]=my_test['F'].shift(i)
    my_test['G_previous_{}'.format(i)]=my_test['G'].shift(i)

C:\Users\karth\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\karth\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\karth\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [368]:
#Getting the Last row alone for each column
my_train = my_train[my_train['record_type'] == 1]
my_test=my_test.groupby('customer_ID').last()
print(my_train.shape)
print(my_test.shape)

(97009, 309)
(55716, 308)


In [369]:
for i in range(1,3):
    my_test['A_previous_{}'.format(i)]=my_test['A_previous_{}'.format(i)].fillna(99)
    my_test['B_previous_{}'.format(i)]=my_test['B_previous_{}'.format(i)].fillna(99)
    my_test['C_previous_{}'.format(i)]=my_test['C_previous_{}'.format(i)].fillna(99)
    my_test['D_previous_{}'.format(i)]=my_test['D_previous_{}'.format(i)].fillna(99)
    my_test['E_previous_{}'.format(i)]=my_test['E_previous_{}'.format(i)].fillna(99)
    my_test['F_previous_{}'.format(i)]=my_test['F_previous_{}'.format(i)].fillna(99)
    my_test['G_previous_{}'.format(i)]=my_test['G_previous_{}'.format(i)].fillna(99)
    
for i in range(1,3):
    my_train['A_previous_{}'.format(i)]=my_train['A_previous_{}'.format(i)].fillna(99)
    my_train['B_previous_{}'.format(i)]=my_train['B_previous_{}'.format(i)].fillna(99)
    my_train['C_previous_{}'.format(i)]=my_train['C_previous_{}'.format(i)].fillna(99)
    my_train['D_previous_{}'.format(i)]=my_train['D_previous_{}'.format(i)].fillna(99)
    my_train['E_previous_{}'.format(i)]=my_train['E_previous_{}'.format(i)].fillna(99)
    my_train['F_previous_{}'.format(i)]=my_train['F_previous_{}'.format(i)].fillna(99)
    my_train['G_previous_{}'.format(i)]=my_train['G_previous_{}'.format(i)].fillna(99)

In [370]:
my_test['customer_ID']=my_test.index
my_test=my_test.reset_index(drop=True)

In [371]:
#One hot encoding 
#options = ['A','B','C','D','E','F','G','A_previous_1','A_previous_2','A_previous_3','B_previous_1','B_previous_2','B_previous_3','C_previous_1','C_previous_2','C_previous_3','D_previous_1','D_previous_2','D_previous_3','E_previous_1','E_previous_2','E_previous_3','F_previous_1','F_previous_2','F_previous_3','G_previous_1','G_previous_2','G_previous_3']
options = ['A','B','C','D','E','F','G','A_previous_1','A_previous_2','B_previous_1','B_previous_2','C_previous_1','C_previous_2','D_previous_1','D_previous_2','E_previous_1','E_previous_2','F_previous_1','F_previous_2','G_previous_1','G_previous_2']


frames = [my_train, my_test]
my_data = pd.concat(frames,keys=['train', 'test'])
print(my_data.shape)
my_data = one_hot(my_data,options)

my_train=my_data.loc['train']
my_test=my_data.loc['test']
print(my_train.shape)
print(my_test.shape)

C:\Users\karth\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


(152725, 309)
(97009, 361)
(55716, 361)


In [372]:
my_train=my_train.merge(purchased_data, left_on='customer_ID', right_on='customer_ID', how='left')

### Prediction of G

In [373]:
#splitting my train data in to train & validation
y = my_train['G_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

Training data size (77607, 359)
validation data size (19402, 359)


In [374]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)
y_pred_prob_val=logreg.predict_proba(X_val) 
metrics.confusion_matrix(y_val,y_pred_val)

array([[4047,    0,    4,    0],
       [   0, 7556,    0,    0],
       [   0,    0, 6140,    0],
       [   0,    0,    0, 1655]], dtype=int64)

In [375]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00      4051
          2       1.00      1.00      1.00      7556
          3       1.00      1.00      1.00      6140
          4       1.00      1.00      1.00      1655

avg / total       1.00      1.00      1.00     19402

0.9997938356870426


In [376]:
#Applying the model on test Data
G_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_G=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
G_pred['G_predicted']=logreg.predict(my_test_G)
# counts=G_pred.groupby('customer_ID').count()
# counts=counts.rename({'G_predicted': 'counts'}, axis=1) 
# G_pred=G_pred.merge(counts, left_on='customer_ID', right_index=True, how='left')
# # G_predicted_mode=G_pred.groupby('customer_ID')['G_predicted'].apply(lambda x: x.mode().iloc[0])
# G_predicted_mode=G_pred.groupby('customer_ID')['G_predicted'].last()

# G_predicted_mode=G_predicted_mode.to_frame()
# G_predicted_mode.reset_index(level=0, inplace=True)

In [377]:
G_pred

customer_ID  G_predicted
0         10000001            2
1         10000002            2
2         10000003            2
3         10000004            3
4         10000006            1
...            ...          ...
55711     10152712            1
55712     10152714            3
55713     10152715            1
55714     10152719            2
55715     10152722            3

[55716 rows x 2 columns]

### Prediction of C & D

In [378]:
#MODEL FOR C

#splitting my train data in to train & validation
y = my_train['C_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (77607, 359)
validation data size (19402, 359)
             precision    recall  f1-score   support

          1       0.99      0.99      0.99      5805
          2       1.00      1.00      1.00      4120
          3       0.98      1.00      0.99      7605
          4       0.99      0.93      0.96      1872

avg / total       0.99      0.99      0.99     19402

0.9892279146479744


In [379]:
#Applying the model on test Data
C_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_C=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
C_pred['C_predicted']=logreg.predict(my_test_C)

In [380]:

# C_predicted_mode=C_pred.groupby('customer_ID')['C_predicted'].last()
# # C_predicted_mode=C_pred.groupby('customer_ID')['C_predicted'].apply(lambda x: x.mode().iloc[0])
# C_predicted_mode=C_predicted_mode.to_frame()
# C_predicted_mode.reset_index(level=0, inplace=True)

In [381]:
#MODEL FOR D

#splitting my train data in to train & validation
y = my_train['D_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (77607, 360)
validation data size (19402, 360)
             precision    recall  f1-score   support

          1       1.00      0.96      0.98      2603
          2       0.98      0.99      0.99      4289
          3       1.00      1.00      1.00     12510

avg / total       0.99      0.99      0.99     19402

0.9936089062983198


In [382]:
#Applying the model on test Data
my_test_D=my_test.copy()
D_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_D=my_test_D.merge(C_pred, left_on='customer_ID', right_on='customer_ID', how='left')
my_test_D=my_test_D.drop(['record_type','customer_ID'], axis=1)

In [383]:
logreg = LogisticRegression()
logreg.fit(X, y)
D_pred['D_predicted']=logreg.predict(my_test_D)

# D_predicted_mode=D_pred.groupby('customer_ID')['D_predicted'].last()
# # D_predicted_mode=D_pred.groupby('customer_ID')['D_predicted'].apply(lambda x: x.mode().iloc[0])
# D_predicted_mode=D_predicted_mode.to_frame()
# D_predicted_mode.reset_index(level=0, inplace=True)

### Prediction of A & F

In [384]:
#MODEL FOR A

#splitting my train data in to train & validation
y = my_train['A_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (77607, 359)
validation data size (19402, 359)
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      4270
          1       1.00      1.00      1.00     11974
          2       1.00      1.00      1.00      3158

avg / total       1.00      1.00      1.00     19402

0.9969075353056386


In [385]:
#Applying the model on test Data
A_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_A=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
A_pred['A_predicted']=logreg.predict(my_test_A)

# A_predicted_mode=A_pred.groupby('customer_ID')['A_predicted'].last()
# # A_predicted_mode=A_pred.groupby('customer_ID')['A_predicted'].apply(lambda x: x.mode().iloc[0])
# A_predicted_mode=A_predicted_mode.to_frame()
# A_predicted_mode.reset_index(level=0, inplace=True)

In [386]:
#MODEL FOR F

#splitting my train data in to train & validation
y = my_train['F_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (77607, 360)
validation data size (19402, 360)
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6234
          1       1.00      1.00      1.00      4834
          2       1.00      1.00      1.00      7201
          3       1.00      0.99      1.00      1133

avg / total       1.00      1.00      1.00     19402

0.9968044531491599


In [387]:
#Applying the model on test Data
my_test_F=my_test.copy()
F_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_F=my_test_F.merge(A_pred, left_on='customer_ID', right_on='customer_ID', how='left')
my_test_F=my_test_F.drop(['record_type','customer_ID'], axis=1)

In [388]:
logreg = LogisticRegression()
logreg.fit(X, y)
F_pred['F_predicted']=logreg.predict(my_test_F)

# F_predicted_mode=F_pred.groupby('customer_ID')['F_predicted'].last()
# # F_predicted_mode=F_pred.groupby('customer_ID')['F_predicted'].apply(lambda x: x.mode().iloc[0])
# F_predicted_mode=F_predicted_mode.to_frame()
# F_predicted_mode.reset_index(level=0, inplace=True)

### Prediction of B & E

In [389]:
#MODEL FOR B

#splitting my train data in to train & validation
y = my_train['B_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (77607, 359)
validation data size (19402, 359)
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     10336
          1       1.00      1.00      1.00      9066

avg / total       1.00      1.00      1.00     19402

1.0


In [390]:
#Applying the model on test Data
B_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_B=my_test.drop(['record_type','customer_ID'], axis=1)

logreg = LogisticRegression()
logreg.fit(X, y)
B_pred['B_predicted']=logreg.predict(my_test_B)

# B_predicted_mode=B_pred.groupby('customer_ID')['B_predicted'].last()
# # B_predicted_mode=B_pred.groupby('customer_ID')['B_predicted'].apply(lambda x: x.mode().iloc[0])
# B_predicted_mode=B_predicted_mode.to_frame()
# B_predicted_mode.reset_index(level=0, inplace=True)

In [391]:
#MODEL FOR E

#splitting my train data in to train & validation
y = my_train['E_purchased']
X = my_train.copy()
X.drop(['record_type','customer_ID','A_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)
print("Training data size", X_train.shape)
print("validation data size", X_val.shape)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_val=logreg.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_val,y_pred_val))
print(accuracy_score(y_val, y_pred_val))

Training data size (77607, 360)
validation data size (19402, 360)
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     10488
          1       1.00      1.00      1.00      8914

avg / total       1.00      1.00      1.00     19402

1.0


In [392]:
#Applying the model on test Data
my_test_E=my_test.copy()
E_pred=pd.DataFrame(my_test[['customer_ID']])
my_test_E=my_test_E.merge(B_pred, left_on='customer_ID', right_on='customer_ID', how='left')
my_test_E=my_test_E.drop(['record_type','customer_ID'], axis=1)

In [393]:
logreg = LogisticRegression()
logreg.fit(X, y)
E_pred['E_predicted']=logreg.predict(my_test_E)
# E_predicted_mode=E_pred.groupby('customer_ID')['E_predicted'].last()
# # E_predicted_mode=E_pred.groupby('customer_ID')['E_predicted'].apply(lambda x: x.mode().iloc[0])
# E_predicted_mode=E_predicted_mode.to_frame()
# E_predicted_mode.reset_index(level=0, inplace=True)

In [394]:
F_predicted_mode.F_predicted.unique()

array([3, 2, 0, 1], dtype=int64)

In [395]:
final_predictions_df=A_predicted_mode.merge(B_predicted_mode,on='customer_ID').merge(C_predicted_mode,on='customer_ID').merge(D_predicted_mode,on='customer_ID').merge(E_predicted_mode,on='customer_ID').merge(F_predicted_mode,on='customer_ID').merge(G_predicted_mode,on='customer_ID')


In [396]:
final_predictions_df=A_pred.merge(B_pred,on='customer_ID').merge(C_pred,on='customer_ID').merge(D_pred,on='customer_ID').merge(E_pred,on='customer_ID').merge(F_pred,on='customer_ID').merge(G_pred,on='customer_ID')


In [397]:
for C in final_predictions_df.columns:
    final_predictions_df[C]=final_predictions_df[C].astype(str)
final_predictions_df['plan']=final_predictions_df['A_predicted'] + final_predictions_df['B_predicted'] + final_predictions_df['C_predicted'] + final_predictions_df['D_predicted'] + final_predictions_df['E_predicted'] + final_predictions_df['F_predicted'] + final_predictions_df['G_predicted']

final_predictions_df=final_predictions_df[['customer_ID','plan']]
final_predictions_df.to_csv(r'C:/Users/karth/Documents/Kaggle/Purchase Prediction/third_model_2.csv', index=False)